<a href="https://colab.research.google.com/github/voodoohop/colabasaservice/blob/master/colabs/deep-daze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deep-daze stomp-py --upgrade

In [13]:
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import stomp

WATCH_MEDIA_FILES = ["./*.jpg","./*.mp4"]

conn = stomp.Connection([('b-4e89df1f-8226-4cc4-a518-4e1ac0023c97-1.mq.eu-central-1.amazonaws.com', 61614)], use_ssl=True)
conn.connect('guest', 'iamcolabguest', wait=True)


!rm *.jpg

p = Popen('imagine --iterations=50 --epochs=5 --save-every=5 --save-progress=True --open_folder=False --overwrite=True "A slick high-contrast"', stdout = PIPE, 
        stderr = STDOUT, shell = True)

FOUND_MEDIA_FILES=[]
while True:
  line = p.stdout.readline()#
  if not line: break

  stdout.write(line)
  conn.send(body=line.decode("utf-8"), destination='/topic/colabOut')
  FOUND_MEDIA_FILES=[glob(search_path) for search_path in WATCH_MEDIA_FILES]
  conn.send(body=FOUND_MEDIA_FILES, destination='/topic/colabMediaOut')
conn.disconnect()

loss: -32.22:  26%|██▌       | 13/50 [00:21<00:56,  1.52s/it]


KeyboardInterrupt: ignored

In [ ]:
from tqdm.notebook import trange
from IPython.display import Image, display

from deep_daze import Imagine

TEXT = 'A slick high-contrast ad for an interactive futuristic school with children, teachers with a dark blue background' #@param {type:"string"}
NUM_LAYERS = 32 #@param {type:"number"}
SAVE_EVERY =  10#@param {type:"number"}
IMAGE_WIDTH =  256#@param {type:"number"}
SAVE_PROGRESS = True #@param {type:"boolean"}
LEARNING_RATE = 1e-5 #@param {type:"number"}
ITERATIONS = 1050 #@param {type:"number"}

model = Imagine(
    text = TEXT,
    num_layers = NUM_LAYERS,
    save_every = SAVE_EVERY,
    image_width = IMAGE_WIDTH,
    lr = LEARNING_RATE,
    iterations = ITERATIONS,
    save_progress = SAVE_PROGRESS
)

for epoch in trange(20, desc = 'epochs'):
    for i in trange(ITERATIONS, desc = 'iteration'):
        model.train_step(epoch, i)

        if i % model.save_every != 0:
            continue

        filename = TEXT[:77].replace(' ', '_')
        print("filename",filename)
        image = Image(f'./{filename}.jpg')
        display(image)

In [ ]:
!zip images.zip *dreams*.jpg
!mkdir images
!mv images.zip images

In [ ]:
from glob import glob
import requests
import requests
from tqdm import tqdm
!mkdir -p results
for imgfile in tqdm(glob("./A_slick*.jpg")):
  print("Super resolution of", imgfile)
  
  r = requests.post(
    "https://api.deepai.org/api/waifu2x",
    files={
        'image': open(imgfile, 'rb'),
    },
    headers={'api-key': "blalasd"}
  )
  srurl = r.json()["output_url"]
  with open(f"/content/results/{imgfile}.upsampled.jpg", 'wb') as f:
    f.write(requests.get(srurl).content)
  #!echo curl -F f"\'image=@{imgfile}\' -H api-key: https://api.deepai.org/api/torch-srgan" 

In [ ]:
!mkdir -p images
!ffmpeg -r 20 -y -i results/A_slick%*.jpg -filter:v scale=512:-1 images/env_school_flashy.mp4

In [ ]:
!pip install stomp-py